# Лабораторная работа №5
## Выполнил студент группы БПИ-2303 Гавриленко Владислав

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [1]:
from datetime import datetime as t

def kmp_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text, pattern = text.lower(), pattern.lower()

    def compute_lps(pattern):
        lps = [0] * len(pattern)
        j = 0  
        i = 1  
        while i < len(pattern):
            if pattern[i] == pattern[j]:
                j += 1
                lps[i] = j
                i += 1
            else:
                if j != 0:
                    j = lps[j - 1]
                else:
                    lps[i] = 0
                    i += 1
        return lps

    lps = compute_lps(pattern)
    i = j = 0  
    positions = []  

    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1
            if j == len(pattern):
                positions.append(i - j)
                j = lps[j - 1]
        else:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return positions

text = input("Введите строку: ")
pattern = input("Введите подстроку: ")
case_sensitive = input("Учитывать регистр? (да/нет): ").strip().lower() == "да"

start = t.now()
result = kmp_search(text, pattern, case_sensitive)
end = t.now()

print("Результат поиска (позиции):", result)
print("Время выполнения:", (end - start).total_seconds(), "секунд")


Результат поиска (позиции): [40, 83]
Время выполнения: 0.000111 секунд


#### Упрощенный алгоритм Бойера-Мура

In [3]:
from collections import defaultdict
from datetime import datetime as t

def boyer_moore_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text, pattern = text.lower(), pattern.lower()

    def build_bad_char_table(pattern):
        bad_char = defaultdict(lambda: len(pattern))
        for i in range(len(pattern) - 1):
            bad_char[pattern[i]] = len(pattern) - 1 - i
        return bad_char

    bad_char = build_bad_char_table(pattern)
    positions = []
    i = 0

    while i <= len(text) - len(pattern):
        j = len(pattern) - 1
        while j >= 0 and pattern[j] == text[i + j]:
            j -= 1
        if j < 0:
            positions.append(i)
            i += len(pattern)
        else:
            i += bad_char[text[i + j]]

    return positions

text = input("Введите строку: ")
pattern = input("Введите подстроку: ")
case_sensitive = input("Учитывать регистр? (да/нет): ").strip().lower() == "да"

start = t.now()
result = boyer_moore_search(text, pattern, case_sensitive)
end = t.now()

print("Результат поиска (позиции):", result)
print("Время выполнения:", (end - start).total_seconds(), "секунд")


Результат поиска (позиции): [56]
Время выполнения: 9.5e-05 секунд


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [9]:
from collections import deque

def is_solvable(puzzle):
    inv_count = sum(
        1
        for i in range(len(puzzle))
        for j in range(i + 1, len(puzzle))
        if puzzle[i] and puzzle[j] and puzzle[i] > puzzle[j]
    )
    return inv_count % 2 == 0

def get_neighbors(state):
    neighbors = []
    zero_index = state.index(0)
    row, col = divmod(zero_index, 4)
    moves = {
        "up": (row - 1, col),
        "down": (row + 1, col),
        "left": (row, col - 1),
        "right": (row, col + 1),
    }
    for move, (r, c) in moves.items():
        if 0 <= r < 4 and 0 <= c < 4:
            new_state = state[:]
            swap_idx = r * 4 + c
            new_state[zero_index], new_state[swap_idx] = new_state[swap_idx], new_state[zero_index]
            neighbors.append((new_state, new_state[zero_index]))
    return neighbors

def solve_puzzle(start_state):
    goal_state = list(range(1, 16)) + [0]
    if not is_solvable(start_state):
        return []

    queue = deque([(start_state, [])])
    visited = set()
    visited.add(tuple(start_state))

    while queue:
        state, path = queue.popleft()
        if state == goal_state:
            return path

        for new_state, moved_tile in get_neighbors(state):
            state_tuple = tuple(new_state)
            if state_tuple not in visited:
                visited.add(state_tuple)
                queue.append((new_state, path + [moved_tile]))

    return []

puzzle = list(map(int, input("Введите 16 чисел через пробел: ").split()))
solution = solve_puzzle(puzzle)
if solution:
    print("Решение найдено:", solution)
else:
    print("Решение отсутствует.")


Решение отсутствует.


### Вывод

В ходе выполнения данной лабораторной работы были рассмотрены алгоритмы для обработки информации. После чего сделаны 2 алгоритма для поисков подстрок, а также алгоритм, проверяющий является ли конкретное расположение символов решаемым.